<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary 代码 for 这个 <一个 href="http://mng.bz/orYv">构建 一个 大语言模型 From Scratch</一个> book by <一个 href="https://sebastianraschka.com">Sebastian Raschka</一个><br>
<br>代码 repository: <一个 href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</一个>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<一个 href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></一个>
</td>
</tr>
</table>

# Evaluating Instruction Responses Locally Using 一个 Llama 3 模型 Via Ollama

- 这个 笔记本 uses 一个 8-billion-参数 Llama 3 模型 through ollama to evaluate responses of instruction finetuned LLMs based on 一个 数据集 in JSON format 那个 includes 这个 generated 模型 responses, for 示例:



```python
{
    "instruction": "什么 is 这个 atomic number of helium?",
    "输入": "",
    "输出": "这个 atomic number of helium is 2.",               # <-- 这个 目标 given in 这个 测试 设置
    "模型 1 response": "\nThe atomic number of helium is 2.0.", # <-- Response by 一个 大语言模型
    "模型 2 response": "\nThe atomic number of helium is 3."    # <-- Response by 一个 2nd 大语言模型
},
```

- 这个 代码 doesn't require 一个 GPU 和 runs on 一个 laptop (它 was tested on 一个 M3 MacBook Air)

In [1]:
from importlib.metadata import version

pkgs = ["tqdm",    # Progress bar
        ]

for p in pkgs:
    print(f"{p} version: {version(p)}")

tqdm version: 4.66.4


## Installing Ollama 和 Downloading Llama 3

- Ollama is 一个 application to 运行 LLMs efficiently
- 它 is 一个 wrapper around [llama.cpp](https://github.com/ggerganov/llama.cpp), 哪个 implements LLMs in pure C/C++ to maximize efficiency
- Note 那个 它 is 一个 tool for using LLMs to 生成 text (推理), not 训练 或者 finetuning LLMs
- Prior to running 这个 代码 below, 安装 ollama by visiting [https://ollama.com](https://ollama.com) 和 following 这个 instructions (for instance, clicking on 这个 "Download" button 和 downloading 这个 ollama application for your operating system)

- For macOS 和 Windows users, click on 这个 ollama application 你 downloaded; 如果 它 prompts 你 to 安装 这个 command line usage, say "yes"
- Linux users can 使用 这个 安装 command provided on 这个 ollama website

- In general, before 我们 can 使用 ollama from 这个 command line, 我们 have to either 开始 这个 ollama application 或者 运行 `ollama serve` in 一个 separate terminal

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/ollama-eval/ollama-serve.webp?1">


- With 这个 ollama application 或者 `ollama serve` running, in 一个 different terminal, on 这个 command line, 执行 这个 following command to try out 这个 8-billion-参数 Llama 3 模型 (这个 模型, 哪个 takes up 4.7 GB of storage space, will be automatically downloaded 这个 首先 time 你 执行 这个 command)

```bash
# 8B 模型
ollama 运行 llama3
```


这个 输出 looks like as follows:

```
$ ollama 运行 llama3
pulling manifest 
pulling 6a0746a1ec1a... 100% ▕████████████████▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕████████████████▏  12 KB                         
pulling 8ab4849b038c... 100% ▕████████████████▏  254 B                         
pulling 577073ffcc6c... 100% ▕████████████████▏  110 B                         
pulling 3f8eb4da87fa... 100% ▕████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 
```

- Note 那个 `llama3` refers to 这个 instruction finetuned 8-billion-参数 Llama 3 模型

- Alternatively, 你 can also 使用 这个 larger 70-billion-参数 Llama 3 模型, 如果 your machine supports 它, by replacing `llama3` with `llama3:70b`

- After 这个 download has been completed, 你 will see 一个 command line prompt 那个 allows 你 to chat with 这个 模型

- Try 一个 prompt like "什么 do llamas eat?", 哪个 should 返回 一个 输出 similar to 这个 following:

```
>>> 什么 do llamas eat?
Llamas are ruminant animals, 哪个 means they have 一个 four-chambered 
stomach 和 eat plants 那个 are high in fiber. In 这个 wild, llamas 
typically feed on:
1. Grasses: They love to graze on various types of grasses, including tall 
grasses, wheat, oats, 和 barley.
```

- 你 can 结束 这个 session using 这个 输入 `/bye`

## Using Ollama's REST API

- 现在, 一个 alternative way to interact with 这个 模型 is via its REST API in Python via 这个 following 函数
- Before 你 运行 这个 接下来 cells in 这个 笔记本, make sure 那个 ollama is still running, as described above, via
  - `ollama serve` in 一个 terminal
  - 这个 ollama application
- 接下来, 运行 这个 following 代码 cell to query 这个 模型

- 首先, 让我们 try 这个 API with 一个 simple 示例 to make sure 它 works as intended:

In [2]:
import urllib.request
import json


def query_model(prompt, model="llama3", url="http://localhost:11434/api/chat"):
    # 创建 这个 data payload as 一个 dictionary
    data = {
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "options": {     # Settings below are required for deterministic responses
            "seed": 123,
            "temperature": 0,
            "num_ctx": 2048
        }
    }

    # 转换 这个 dictionary to 一个 JSON formatted string 和 encode 它 to bytes
    payload = json.dumps(data).encode("utf-8")

    # 创建 一个 request object, setting 这个 方法 to POST 和 adding necessary headers
    request = urllib.request.Request(url, data=payload, method="POST")
    request.add_header("Content-Type", "application/json")

    # Send 这个 request 和 capture 这个 response
    response_data = ""
    with urllib.request.urlopen(request) as response:
        # Read 和 decode 这个 response
        while True:
            line = response.readline().decode("utf-8")
            if not line:
                break
            response_json = json.loads(line)
            response_data += response_json["message"]["content"]

    return response_data


result = query_model("What do Llamas eat?")
print(result)

Llamas are herbivores, which means they primarily feed on plant-based foods. Their diet typically consists of:

1. Grasses: Llamas love to graze on various types of grasses, including tall grasses, short grasses, and even weeds.
2. Hay: High-quality hay, such as alfalfa or timothy hay, is a staple in a llama's diet. They enjoy the sweet taste and texture of fresh hay.
3. Grains: Llamas may receive grains like oats, barley, or corn as part of their daily ration. However, it's essential to provide these grains in moderation, as they can be high in calories.
4. Fruits and vegetables: Llamas enjoy a variety of fruits and veggies, such as apples, carrots, sweet potatoes, and leafy greens like kale or spinach.
5. Minerals: Llamas require access to mineral supplements, which help maintain their overall health and well-being.

In the wild, llamas might also eat:

1. Leaves: They'll munch on leaves from trees and shrubs, including plants like willow, alder, and birch.
2. Bark: In some cases, ll

## 加载 JSON Entries

- 现在, 让我们 获取 to 这个 data evaluation part
- 这里, 我们 assume 那个 我们 saved 这个 测试 数据集 和 这个 模型 responses as 一个 JSON file 那个 我们 can 加载 as follows:

In [3]:
json_file = "eval-example-data.json"

with open(json_file, "r") as file:
    json_data = json.load(file)

print("Number of entries:", len(json_data))

Number of entries: 100


- 这个 structure of 这个 file is as follows, 哪里 我们 have 这个 given response in 这个 测试 数据集 (`'输出'`) 和 responses by two different models (`'模型 1 response'` 和 `'模型 2 response'`):

In [4]:
json_data[0]

{'instruction': 'Calculate the hypotenuse of a right triangle with legs of 6 cm and 8 cm.',
 'input': '',
 'output': 'The hypotenuse of the triangle is 10 cm.',
 'model 1 response': '\nThe hypotenuse of the triangle is 3 cm.',
 'model 2 response': '\nThe hypotenuse of the triangle is 12 cm.'}

- Below is 一个 small utility 函数 那个 formats 这个 输入 for visualization purposes later:

In [5]:
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. Write a response that "
        f"appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### 输入:\n{entry['输入']}" 如果 entry["输入"] else ""
    instruction_text + input_text

    return instruction_text + input_text

- 现在, 让我们 try 这个 ollama API to compare 这个 模型 responses (我们 only evaluate 这个 首先 5 responses for 一个 visual comparison):

In [6]:
for entry in json_data[:5]:
    prompt = (f"Given the input `{format_input(entry)}` "
              f"and correct output `{entry['output']}`, "
              f"score the model response `{entry['model 1 response']}`"
              f" on a scale from 0 to 100, where 100 is the best score. "
              )
    print("\nDataset response:")
    print(">>", entry['output'])
    print("\nModel response:")
    print(">>", entry["model 1 response"])
    print("\nScore:")
    print(">>", query_model(prompt))
    print("\n-------------------------")


Dataset response:
>> The hypotenuse of the triangle is 10 cm.

Model response:
>> 
The hypotenuse of the triangle is 3 cm.

Score:
>> I'd score this response as 0 out of 100.

The correct answer is "The hypotenuse of the triangle is 10 cm.", not "3 cm.". The model failed to accurately calculate the length of the hypotenuse, which is a fundamental concept in geometry and trigonometry.

-------------------------

Dataset response:
>> 1. Squirrel
2. Eagle
3. Tiger

Model response:
>> 
1. Squirrel
2. Tiger
3. Eagle
4. Cobra
5. Tiger
6. Cobra

Score:
>> I'd rate this model response as 60 out of 100.

Here's why:

* The model correctly identifies two animals that are active during the day: Squirrel and Eagle.
* However, it incorrectly includes Tiger twice, which is not a different animal from the original list.
* Cobra is also an incorrect answer, as it is typically nocturnal or crepuscular (active at twilight).
* The response does not meet the instruction to provide three different animals

- Note 那个 这个 responses are very verbose; to quantify 哪个 模型 is better, 我们 only want to 返回 这个 scores:

In [7]:
from tqdm import tqdm


def generate_model_scores(json_data, json_key):
    scores = []
    for entry in tqdm(json_data, desc="Scoring entries"):
        prompt = (
            f"Given the input `{format_input(entry)}` "
            f"and correct output `{entry['output']}`, "
            f"score the model response `{entry[json_key]}`"
            f" on a scale from 0 to 100, where 100 is the best score. "
            f"Respond with the integer number only."
        )
        score = query_model(prompt)
        try:
            scores.append(int(score))
        except ValueError:
            continue

    return scores

- 让我们 现在 应用 这个 evaluation to 这个 whole 数据集 和 计算 这个 average score of each 模型 (这个 takes about 1 minute per 模型 on 一个 M3 MacBook Air laptop)
- Note 那个 ollama is not fully deterministic across operating systems (as of 这个 writing) so 这个 numbers 你 are getting might slightly differ from 这个 ones shown below

In [8]:
from pathlib import Path

for model in ("model 1 response", "model 2 response"):

    scores = generate_model_scores(json_data, model)
    print(f"\n{model}")
    print(f"Number of scores: {len(scores)} of {len(json_data)}")
    print(f"Average score: {sum(scores)/len(scores):.2f}\n")

    # Optionally 保存 这个 scores
    save_path = Path("scores") / f"llama3-8b-{model.replace(' ', '-')}.json"
    with open(save_path, "w") as file:
        json.dump(scores, file)

Scoring entries: 100%|████████████████████████| 100/100 [01:02<00:00,  1.59it/s]



model 1 response
Number of scores: 100 of 100
Average score: 78.48



Scoring entries: 100%|████████████████████████| 100/100 [01:10<00:00,  1.42it/s]


model 2 response
Number of scores: 99 of 100
Average score: 64.98



- Based on 这个 evaluation above, 我们 can say 那个 这个 1st 模型 is better than 这个 2nd 模型